## NumPy and SciPy in Action

In this section we'll use NumPy and SciPy to write a custom data analysis algorithm for the data we just cleaned. We'll use Friedman's "Projection Pursuit Regression" (PPR) algorithm which is sort of like a souped up neural network. There are no widely used existing implementations of PPR for Python so we can't take the easy way out and use SciKit learn.

PPR is a generalization of a "generalized additive model" (GAM) which expresses a function as a linear combination of smooth functions of a set of predictors. PPR extends the GAM to allow nonlinear relationships between predictors. Intuitively it can be thought of as iteratively finding the "directions" in the data which explain the most variance.

PPR approximates a function as:

$$y = \sum_{h=1}^{H}g_{h}(w_{h}^{T}x)$$

where the "g" are smooth nonparameteric functions (typically a spline). PPR and it's more widely known relative, the multilayer perceptron, are part of a larger family of approximation techniques based on "ridge functions." PPR has several advantages which make it generally preferable to an MLP:

1. Generally requires far fewer hidden units to approximate a funtion to the same level of accuracy
2. Simpler algorithm for estimation
3. Much less hyperparameter tuning

The model is estimated using a straightforward algorithm based on iteratively reweighted least squares. Let's just consider fitting a single ridge function first and assume that we're minimizing L2 norm. Let `X` be an `R^{N x K}` matrix of data and let `y` be an `N` vector of outcomes in `R`.

1. Initialize a vector of weights `w = random(k,1)`
2. Compute `v = Xw`
3. Estimate the spline `g(v)` minimizing `||y - g(v)||`. We can either use a smoothing spline, or choose the knots by hand. I've found it's best to choose knots at quantiles of `v`.
4. Update the vector of weights as the solution to a weighted least squares problem: `w = inv(t(X)WX)(t(X)Wm)`. `W` is an `N x N` diagonal matrix such that `diag(W)[i] = g'(v[i])^2` and `m` is an `N` vector: `m = v + ((y - g(v)) / (g'(v))` where `g'(v)` denotes the first derivative of `g` evaluated at `v`.

Then to add more directions we simply keep applying the algorithm above until error stops decreasing. The only change is that instead of `y` we use the residuals after previous terms. Fun fact: this method will always yield the model with lowest residual variance in the limit!


In [72]:
import numpy as np
import numpy.linalg as alg
import scipy.interpolate as ipl
import pandas as pd
import matplotlib.pyplot as plt

In [67]:
class PPRNode:
    """
    Simple container to hold the parameters associated with a PPR node
    """

    def __init__(self, weights=None, spline=None, mu=0):
        self.weights = weights
        self.spline = spline
        self.mu = 0
        
class ProjectionPursuitRegressor:
    def __init__(self, knots=2, max_terms=15, train_split=0.8,
                 tolerance=1.0, inner_iter_tol=1e-5):
        self._smoother = self._least_squares_spline
        self._knots = knots
        self._params = []
        self._test_mse = []
        self._train_mse = []
        self._train_split = train_split
        self._max_terms = max_terms
        self._outer_iter_tol = tolerance
        self._inner_iter_tol = inner_iter_tol
        self._stabilizer = 3.2
        
    def fit(self, X, y):
        # A common "gotcha" with NumPy involves accidentally
        # reshaping two arrays into a matrix through broadcasting.
        # Unraveling vectors helps avoid this
        y = y.ravel()
        
        # Let's split our input data into a training and validation set
        split = self._train_test_split(X, y, self._train_split)
        X_train, y_train, X_test, y_test = split

        # now we actually run the optimization
        num_terms = 0
        mse = np.inf
        residuals = y_train
        while num_terms < self._max_terms:            
            # This call adds a new term to the model
            self._fit_term(X_train, residuals)
            
            # now recompute goodness of fit metrics
            mse_new_train = self._compute_mse(X_train, y_train)
            mse_new_test = self._compute_mse(X_test, y_test)
            self._train_mse.append(mse_new_train)
            self._test_mse.append(mse_new_test)
            
            # check to see if we're within the tolerance for convergence
            if np.abs(mse_new_train - mse) < self._outer_iter_tol:
                break
            
            # otherwise add a new term and continue
            residuals = y_train - self.predict(X_train)
            mse = mse_new_train
            num_terms += 1
            
            msg = 'Term: {} fit. Train MSE: {} => Test MSE: {}'
            print msg.format(num_terms, mse_new_train, mse_new_test)
        
        msg = 'Converged! Final Train MSE: {} => Test MSE: {}'
        print msg.format(mse_new_train, mse_new_test)
        # now pare back the model to the one which gave best test performance
        self._params = self._params[:np.argmin(self._test_mse)+1]
    
    def _fit_term(self, X, y):
        """
        Internal workhorse function to fit an individual term using
        iteratively reweighted least squares.
        """

        node = PPRNode()
        self._params.append(node)
        
        # initialize the weights with uniform random values
        w = np.random.rand(X.shape[1],1).ravel()
        
        converged = False
        cost = np.Inf
        iteration = 0
        while not converged:
            # solve the weight update equations
            v = X.dot(w)
            g, g_prime, spline = self._smoother(v, y)
            eps = y - g                                
            g_prime_squared = np.power(g_prime,2)   
            m = v + np.divide(eps, g_prime)            
            w = self._wls(X, g_prime_squared, m)

            node.spline = spline                       
            node.weights = w

            # recompute goodness of fit and check if the term has converged
            cost_new = self._compute_mse(X, y)
            if np.abs(cost-cost_new) < self._inner_iter_tol:
                converged = True
            if iteration > 100:
                break
            cost = cost_new
            iteration += 1
            
    def _wls(self, X, w, y):
        """
        Internal method to fit a weighted least squares model
        """

        XTW = X.T*w
        return self._solve_linear_system(XTW.dot(X), XTW.dot(y))

    def _solve_linear_system(self, A, b):
        """
        Solves a system of equations Ax = b using ridge regularization
        """

        AA = A.copy()
        np.fill_diagonal(AA, self._stabilizer+np.diag(A))
        return alg.solve(AA, b)
    
    def _least_squares_spline(self, v, y):
        """
        Fits y = g(v) where g is a cubic spline with the stipulated knots. 
        """
        ixs = np.argsort(v)
        _v = v[ixs]
        _y = y[ixs]
        
        # sometimes the values end up too close together due to numerical
        # imprecision. If this happens just jitter them a bit.
        # it'll all shake out later.
        if not np.all(np.diff(_v) > 0.0):
            v = v + np.random.uniform(1e-9, 1e-8, v.shape)
            ixs = np.argsort(v)
            _v = v[ixs]
            _y = y[ixs]
            
        knots_actual = np.percentile(_v, np.linspace(0, 100, self._knots))
        if np.abs(knots_actual[0] - _v[0]) < 1e-6:
            knots_actual = knots_actual[1:]
        if np.abs(knots_actual[-1] - _v[-1]) < 1e-6:
            knots_actual = knots_actual[:-1]        
        spline = ipl.LSQUnivariateSpline(_v, _y, knots_actual, k=3)
        deriv = spline.derivative()
        return (spline(v), deriv(v), spline)
            
    def _compute_mse(self, X, y):
        """
        Compute the goodness of fit for this model (as measured by MSE)
        """

        y_hat = self.predict(X)
        return np.mean(np.power(y - y_hat, 2))
    
    def predict(self, X, leave_out=None):
        """
        Internal workhorse function to form predicted values for the current
        state of the model
        """

        directions = np.array([p.spline(X.dot(p.weights)) for p in self._params])
        y = np.zeros(directions[0].shape)
        for direction in directions:
            y = y + direction
        return y
        
    @staticmethod
    def _train_test_split(X, y, split_pct):
        is_test_set = (np.random.rand(X.shape[0],1) > split_pct).ravel()
        X_test = X[is_test_set,:]
        y_test = y[is_test_set]
        X_train = X[np.logical_not(is_test_set),:]
        y_train = y[np.logical_not(is_test_set)]
        return X_train, y_train, X_test, y_test

## Now let's see our model in action on some synthetic and real data.

We'll show that our model is capable of learning an interaction term and then look at it for our real data set.

In [68]:
np.random.seed(19832)

# We'll generate a synthetic function with a nonlinear interaction
# between two variables and then jitter it with gaussian white noise
X = np.random.rand(1000,10)*100
XR = np.concatenate((np.ones((1000,1)), X.copy()), axis=1)
X = np.concatenate((np.ones((1000,1)), X, (X[:,1]*X[:,2]).reshape(1000,1),
                   (X[:,1]*X[:,3]).reshape(1000,1)), axis=1)
y = X.dot(np.random.rand(X.shape[1],1)) + np.random.normal(0,1,(1000,1))

# we should rescale our data to have zero mean and unit variance
# note how broadcasting makes this really convenient
X = (X[:,1:] - X[:,1:].mean(axis=0)) / np.sqrt(X[:,1:].var(axis=0))
XR = (XR[:,1:] - XR[:,1:].mean(axis=0)) / np.sqrt(XR[:,1:].var(axis=0))

P = ProjectionPursuitRegressor()
P.fit(XR, y)

Term: 1 fit. Train MSE: 36811.0915102 => Test MSE: 38012.5584023
Term: 2 fit. Train MSE: 3270.76293224 => Test MSE: 3415.73706242
Term: 3 fit. Train MSE: 519.753696355 => Test MSE: 569.185960829
Term: 4 fit. Train MSE: 266.624001123 => Test MSE: 272.376870423
Term: 5 fit. Train MSE: 44.1814674237 => Test MSE: 46.1097368758
Term: 6 fit. Train MSE: 15.7774777042 => Test MSE: 14.6214671089
Term: 7 fit. Train MSE: 10.602041946 => Test MSE: 11.3277039963
Term: 8 fit. Train MSE: 8.55168639181 => Test MSE: 8.08262752046
Term: 9 fit. Train MSE: 6.68101197363 => Test MSE: 7.51201681698
Term: 10 fit. Train MSE: 3.97409418426 => Test MSE: 5.23783807172
Converged! Final Train MSE: 3.0406406737 => Test MSE: 3.82161850765


In [73]:
# Now let's give it a try on our real dataset
air = pd.read_csv('clean_air_data.csv')
air = air.set_index(pd.to_datetime(air['timestamp'])).sort_index()

cols_to_drop = filter(lambda x: 'PT08' in x, air.columns)
air = air.drop(cols_to_drop, axis='columns')

# our data is a time-series so we'll model it using a finite distributed lag
# with three periods
# and include fixed-effects for month of year and AM vs. PM

air['month'] = air.index.map(lambda x: x.month)
air['is_morning'] = air.index.map(lambda x: x.hour < 12)
fe_cols = pd.get_dummies(
        air['month'], drop_first=True
    ).join(air['is_morning'].astype(np.int32), how='inner')

y = air['CO(GT)'].values.ravel()[3:]
depvars = air.drop(['timestamp','CO(GT)','month','is_morning'], axis='columns').sort_index()
X = depvars.join(
        depvars.shift(1), how='inner', rsuffix='_L1'
    ).join(
        depvars.shift(2), how='inner', rsuffix='_L2'
    ).join(
        depvars.shift(3), how='inner', rsuffix='_L3'
    ).dropna()

# now let's also rescale this to zero mean
X = (X - X.mean(axis=0)) / np.sqrt(X.var(axis=0))

# and lastly join on the fixed effects
X = X.join(fe_cols, how='inner')
print X.head()

                     NMHC(GT)  C6H6(GT)   NOx(GT)   NO2(GT)         T  \
timestamp                                                               
2004-03-10 21:00:00  0.826228  0.263163  0.626631  0.797512  0.089890   
2004-03-10 22:00:00  1.532394  0.117622 -0.154504  0.467118  0.036331   
2004-03-10 23:00:00  1.437413  0.072096 -0.321908  0.304744  0.036331   
2004-03-11 00:00:00  1.386269  0.044274 -0.429525  0.150489  0.038751   
2004-03-11 01:00:00  1.386269  0.036686 -0.429525  0.142371  0.024231   

                           RH        AH  NMHC(GT)_L1  C6H6(GT)_L1  NOx(GT)_L1  \
timestamp                                                                       
2004-03-10 21:00:00  0.246092  0.207562     1.802197     0.180861   -0.154448   
2004-03-10 22:00:00  0.411657  0.206929     0.825863     0.263172    0.626692   
2004-03-10 23:00:00  0.403492  0.206822     1.531911     0.117630   -0.154448   
2004-03-11 00:00:00  0.354502  0.206163     1.436945     0.072104   -0.321853   
20

In [74]:
# Now let's compare PPR and a vanilla least squares reression
# Both of these are actually terrible models for this dataset
# but it's illustrative of how to use NumPy so let's just go with it...

X = X.values

# Let's also partition our data into a train and test set
is_test = np.random.rand(X.shape[0],1).ravel() > 0.80
X_test = X[is_test,:]
y_test = y[is_test]
X_train = X[np.logical_not(is_test),:]
y_train = y[np.logical_not(is_test)]

b = alg.solve(X_train.T.dot(X_train), X_train.T.dot(y_train))
y_hat = X_test.dot(b)
eps = y_test - y_hat
print 'MSE (OLS): {}'.format(np.power(eps,2).mean())

MSE (OLS): 2445.27952118


In [75]:
np.random.seed(13298)
P = ProjectionPursuitRegressor()
P.fit(X_train, y_train)
y_hat = P.predict(X_test)

eps = y_test - y_hat
print 'MSE (PPR): {}'.format(np.power(eps,2).mean())

Term: 1 fit. Train MSE: 2034.55162507 => Test MSE: 2226.96726137
Term: 2 fit. Train MSE: 1912.92038718 => Test MSE: 2135.45174834
Term: 3 fit. Train MSE: 1828.67961698 => Test MSE: 2036.7297136
Term: 4 fit. Train MSE: 1769.67349769 => Test MSE: 2060.2256412
Term: 5 fit. Train MSE: 1734.68225858 => Test MSE: 2016.76803863
Term: 6 fit. Train MSE: 1714.23969093 => Test MSE: 2009.55890516
Term: 7 fit. Train MSE: 1690.48480444 => Test MSE: 2013.08279575
Term: 8 fit. Train MSE: 1672.91337456 => Test MSE: 2015.56052184
Term: 9 fit. Train MSE: 1644.69128253 => Test MSE: 2003.18421334
Term: 10 fit. Train MSE: 1628.83548853 => Test MSE: 1996.69945616
Term: 11 fit. Train MSE: 1614.24884996 => Test MSE: 1987.56259109
Term: 12 fit. Train MSE: 1591.28778643 => Test MSE: 1987.03894426
Term: 13 fit. Train MSE: 1575.56993104 => Test MSE: 1986.09438947
Term: 14 fit. Train MSE: 1534.23810555 => Test MSE: 1971.27269044
Term: 15 fit. Train MSE: 1526.08589199 => Test MSE: 1963.47666131
Converged! Final Trai